# Введение в my_map_reduce модель на Python


In [1]:
from typing import Iterator
from typing import NamedTuple

In [2]:
class User(NamedTuple):
    id: int
    age: int
    social_contacts: int
    gender: str

In [3]:
def my_map(_, row: User):
    if row.gender == 'female':
        yield row.age, row


def my_reduce(age: str, rows: Iterator[NamedTuple]):
    sum = 0
    count = 0
    for row in rows:
        sum += row.social_contacts
        count += 1
    if count > 0:
        yield age, sum / count
    else:
        yield age, 0

Модель элемента данных

In [4]:
input_collection = [
    User(id=0, age=55, gender='male', social_contacts=20),
    User(id=1, age=25, gender='female', social_contacts=240),
    User(id=2, age=25, gender='female', social_contacts=500),
    User(id=3, age=33, gender='female', social_contacts=800)
]

Функция record_reader моделирует чтение элементов с диска или по сети.

In [5]:
def record_reader():
    return [(u.id, u) for u in input_collection]

In [6]:
list(record_reader())

[(0, User(id=0, age=55, social_contacts=20, gender='male')),
 (1, User(id=1, age=25, social_contacts=240, gender='female')),
 (2, User(id=2, age=25, social_contacts=500, gender='female')),
 (3, User(id=3, age=33, social_contacts=800, gender='female'))]

In [7]:
def flatten(nested_iterable):
    for iterable in nested_iterable:
        for element in iterable:
            yield element

In [8]:
map_output = flatten(map(lambda x: my_map(*x), record_reader()))
map_output = list(map_output)  # materialize
map_output

[(25, User(id=1, age=25, social_contacts=240, gender='female')),
 (25, User(id=2, age=25, social_contacts=500, gender='female')),
 (33, User(id=3, age=33, social_contacts=800, gender='female'))]

In [9]:
def group_by_key(iterable):
    t = {}
    for (k2, v2) in iterable:
        t[k2] = t.get(k2, []) + [v2]
    return t.items()

In [10]:
shuffle_output = group_by_key(map_output)
shuffle_output = list(shuffle_output)
shuffle_output

[(25,
  [User(id=1, age=25, social_contacts=240, gender='female'),
   User(id=2, age=25, social_contacts=500, gender='female')]),
 (33, [User(id=3, age=33, social_contacts=800, gender='female')])]

In [11]:
reduce_output = flatten(map(lambda x: my_reduce(*x), shuffle_output))
reduce_output = list(reduce_output)
reduce_output

[(25, 370.0), (33, 800.0)]

Все действия одним конвейером!

In [12]:
list(flatten(map(lambda x: my_reduce(*x), group_by_key(flatten(map(lambda x: my_map(*x), record_reader()))))))

[(25, 370.0), (33, 800.0)]

# **my_map_reduce**
Выделим общую для всех пользователей часть системы в отдельную функцию высшего порядка. Это наиболее простая модель my_map_reduce, без учёта распределённого хранения данных.

Пользователь для решения своей задачи реализует record_reader, my_map, my_reduce.

In [13]:
def flatten(nested_iterable):
    for iterable in nested_iterable:
        for element in iterable:
            yield element


def group_by_key(iterable):
    t = {}
    for (k2, v2) in iterable:
        t[k2] = t.get(k2, []) + [v2]
    return t.items()


def my_map_reduce(record_reader, my_map, my_reduce):
    return flatten(map(lambda x: my_reduce(*x), group_by_key(flatten(map(lambda x: my_map(*x), record_reader())))))

## Спецификация my_map_reduce



```
f (k1, v1) -> (k2,v2)*
g (k2, v2*) -> (k3,v3)*

mapreduce ((k1,v1)*) -> (k3,v3)*
groupby ((k2,v2)*) -> (k2,v2*)*
flatten (e2**) -> e2*

mapreduce .map(f).flatten.groupby(k2).map(g).flatten
```




# Примеры

## SQL

In [14]:
from typing import NamedTuple  # requires python 3.6+
from typing import Iterator


class User(NamedTuple):
    id: int
    age: int
    social_contacts: int
    gender: str


input_collection = [
    User(id=0, age=55, gender='male', social_contacts=20),
    User(id=1, age=25, gender='female', social_contacts=240),
    User(id=2, age=25, gender='female', social_contacts=500),
    User(id=3, age=33, gender='female', social_contacts=800)
]


def my_map(_, row: User):
    if row.gender == 'female':
        yield row.age, row


def my_reduce(age: str, rows: Iterator[NamedTuple]):
    sum = 0
    count = 0
    for row in rows:
        sum += row.social_contacts
        count += 1
    if count > 0:
        yield age, sum / count
    else:
        yield age, 0


def record_reader():
    return [(u.id, u) for u in input_collection]


output = my_map_reduce(record_reader, my_map, my_reduce)
output = list(output)
output

[(25, 370.0), (33, 800.0)]

## Matrix-Vector multiplication

In [15]:
from typing import Iterator
import numpy as np

mat = np.ones((5, 4))
vec = np.random.rand(4)  # in-memory vector in all map tasks


def my_map(coordinates: (int, int), value: int):
    i, j = coordinates
    yield i, value * vec[j]


def my_reduce(i: int, products: Iterator[NamedTuple]):
    sum = 0
    for p in products:
        sum += p
    yield i, sum


def record_reader():
    for i in range(mat.shape[0]):
        for j in range(mat.shape[1]):
            yield (i, j), mat[i, j]


output = my_map_reduce(record_reader, my_map, my_reduce)
output = list(output)
output

[(0, np.float64(1.742220737469483)),
 (1, np.float64(1.742220737469483)),
 (2, np.float64(1.742220737469483)),
 (3, np.float64(1.742220737469483)),
 (4, np.float64(1.742220737469483))]

## Inverted index

In [16]:
from typing import Iterator

d1 = "it is what it is"
d2 = "what is it"
d3 = "it is a banana"
documents = [d1, d2, d3]


def record_reader():
    for (docid, document) in enumerate(documents):
        yield "{}".format(docid), document


def my_map(doc_id: str, body: str):
    for word in set(body.split(' ')):
        yield word, doc_id


def my_reduce(word: str, doc_ids: Iterator[str]):
    yield word, sorted(doc_ids)


output = my_map_reduce(record_reader, my_map, my_reduce)
output = list(output)
output

[('it', ['0', '1', '2']),
 ('is', ['0', '1', '2']),
 ('what', ['0', '1']),
 ('banana', ['2']),
 ('a', ['2'])]

## WordCount

In [17]:
from typing import Iterator

d1 = """
it is what it is
it is what it is
it is what it is"""
d2 = """
what is it
what is it"""
d3 = """
it is a banana"""
documents = [d1, d2, d3]


def record_reader():
    for (docid, document) in enumerate(documents):
        for (lineid, line) in enumerate(document.split('\n')):
            yield "{}:{}".format(docid, lineid), line


def my_map(docId: str, line: str):
    for word in line.split(" "):
        yield word, 1


def my_reduce(word: str, counts: Iterator[int]):
    sum = 0
    for c in counts:
        sum += c
    yield word, sum


output = my_map_reduce(record_reader, my_map, my_reduce)
output = list(output)
output

[('', 3), ('it', 9), ('is', 9), ('what', 5), ('a', 1), ('banana', 1)]

# my_map_reduce Distributed

Добавляется в модель фабрика RECORDREARER-ов --- input_format, функция распределения промежуточных результатов по партициям partitioner, и функция COMBINER для частичной аггрегации промежуточных результатов до распределения по новым партициям.

In [18]:
def flatten(nested_iterable):
    for iterable in nested_iterable:
        for element in iterable:
            yield element


def group_by_key(iterable):
    t = {}
    for (k2, v2) in iterable:
        t[k2] = t.get(k2, []) + [v2]
    return t.items()


def groupbykey_distributed(map_partitions, partitioner):
    global reducers
    partitions = [dict() for _ in range(reducers)]
    for map_partition in map_partitions:
        for (k2, v2) in map_partition:
            p = partitions[partitioner(k2)]
            p[k2] = p.get(k2, []) + [v2]
    return [(partition_id, sorted(partition.items(), key=lambda x: x[0])) for (partition_id, partition) in
            enumerate(partitions)]


def partitioner(obj):
    global reducers
    return hash(obj) % reducers


def map_reduce_distributed(input_format, my_map, my_reduce, partitioner=partitioner, combiner=None):
    map_partitions = map(lambda record_reader: flatten(map(lambda k1v1: my_map(*k1v1), record_reader)), input_format())

    if combiner is not None:
        map_partitions = map(lambda map_partition:
                             flatten(map(lambda k2v2: combiner(*k2v2), group_by_key(map_partition))), map_partitions)

    reduce_partitions = groupbykey_distributed(map_partitions, partitioner)  # shuffle
    reduce_outputs = map(lambda reduce_partition:
                         (reduce_partition[0],
                          flatten(map(lambda reduce_input_group:
                                      my_reduce(*reduce_input_group), reduce_partition[1]))), reduce_partitions)

    print("{} key-value pairs were sent over a network.".format(
        sum([len(vs) for (k, vs) in
             flatten([partition for (partition_id, partition) in reduce_partitions])])))

    return reduce_outputs

## Спецификация my_map_reduce Distributed


```
f (k1, v1) -> (k2,v2)*
g (k2, v2*) -> (k3,v3)*

e1 (k1, v1)
e2 (k2, v2)
partition1 (k2, v2)*
partition2 (k2, v2*)*

flatmap (e1->e2*, e1*) -> partition1*
groupby (partition1*) -> partition2*

mapreduce ((k1,v1)*) -> (k3,v3)*
mapreduce .flatmap(f).groupby(k2).flatmap(g)
```



## WordCount

In [19]:
from typing import Iterator

d1 = """
it is what it is
it is what it is
it is what it is"""
d2 = """
what is it
what is it"""
d3 = """
it is a banana"""
documents = [d1, d2, d3, d1, d2, d3]

maps = 3
reducers = 2


def input_format():
    global maps

    def record_reader(split):
        for (docid, document) in enumerate(split):
            for (lineid, line) in enumerate(document.split('\n')):
                yield "{}:{}".format(docid, lineid), line

    split_size = int(np.ceil(len(documents) / maps))
    for i in range(0, len(documents), split_size):
        yield record_reader(documents[i:i + split_size])


def my_map(docId: str, line: str):
    for word in line.split(" "):
        yield word, 1


def my_reduce(word: str, counts: Iterator[int]):
    sum = 0
    for c in counts:
        sum += c
    yield word, sum


# try to set COMBINER=REDUCER and look at the number of values sent over the network
partitioned_output = map_reduce_distributed(input_format, my_map, my_reduce, combiner=None)
partitioned_output = [(partition_id, list(partition)) for (partition_id, partition) in partitioned_output]
partitioned_output

56 key-value pairs were sent over a network.


[(0, [('', 6), ('banana', 2), ('is', 18), ('it', 18)]),
 (1, [('a', 2), ('what', 10)])]

## TeraSort

In [20]:
import numpy as np

input_values = np.random.rand(30)
maps = 3
reducers = 2
min_value = 0.0
max_value = 1.0


def input_format():
    global maps

    def record_reader(split):
        for value in split:
            yield value, None

    split_size = int(np.ceil(len(input_values) / maps))
    for i in range(0, len(input_values), split_size):
        yield record_reader(input_values[i:i + split_size])


def my_map(value: int, _):
    yield value, None


def partitioner(key):
    global reducers
    global max_value
    global min_value
    bucket_size = (max_value - min_value) / reducers
    bucket_id = 0
    while (key > (bucket_id + 1) * bucket_size) and ((bucket_id + 1) * bucket_size < max_value):
        bucket_id += 1
    return bucket_id


def my_reduce(value: int, _):
    yield None, value


partitioned_output = map_reduce_distributed(input_format, my_map, my_reduce, combiner=None, partitioner=partitioner)
partitioned_output = [(partition_id, list(partition)) for (partition_id, partition) in partitioned_output]
partitioned_output

30 key-value pairs were sent over a network.


[(0,
  [(None, np.float64(0.022566506217998494)),
   (None, np.float64(0.11183061640214509)),
   (None, np.float64(0.11922751479960814)),
   (None, np.float64(0.13744666757200008)),
   (None, np.float64(0.13841118544520625)),
   (None, np.float64(0.1388860785768995)),
   (None, np.float64(0.15741275626144602)),
   (None, np.float64(0.15988108632682096)),
   (None, np.float64(0.1805433015933614)),
   (None, np.float64(0.18080343507309404)),
   (None, np.float64(0.2499028063670572)),
   (None, np.float64(0.2938295823738537)),
   (None, np.float64(0.29708762514938825)),
   (None, np.float64(0.30011224970796446)),
   (None, np.float64(0.30533032755860545)),
   (None, np.float64(0.35031309540844435)),
   (None, np.float64(0.3614859964416267)),
   (None, np.float64(0.3628712403828649)),
   (None, np.float64(0.42900034790454855))]),
 (1,
  [(None, np.float64(0.5370053080266557)),
   (None, np.float64(0.5564309682878759)),
   (None, np.float64(0.7218870047875557)),
   (None, np.float64(0.73433

# Упражнения
Упражнения взяты из Rajaraman A., Ullman J. D. Mining of massive datasets. – Cambridge University Press, 2011.


Для выполнения заданий переопределите функции record_reader, my_map, my_reduce. Для модели распределённой системы может потребоваться переопределение функций PARTITION и COMBINER.

### Максимальное значение ряда

Разработайте my_map_reduce алгоритм, который находит максимальное число входного списка чисел.

In [21]:
from typing import Iterator

# ─ входной «поток» значений
input = [42, -7, 19, 23, 58, 99, 12, 0, -3, 77]


def record_reader():
    """
    Последовательно отдаём элементы в формате (ключ, значение).
    Ключом здесь служит строковый индекс, а значением — само число.
    """
    for idx, num in enumerate(input):
        yield str(idx), num


def my_map(num_id: str, num: int):
    """
    Отдаём все записи в один и тот же раздел (ключ 0),
    чтобы редьюсер видел вообще все числа.
    """
    yield 0, num


def my_reduce(dummy_key: str, nums: Iterator[int]):
    """
    Получаем все числа и выводим их максимум.
    dummy_key всегда '0', потому что my_map отдаёт ровно один ключ.
    """
    yield dummy_key, max(nums)


# Проверяем работу алгоритма
print("MAX:", list(my_map_reduce(record_reader, my_map, my_reduce)))  # [('0', 99)]

MAX: [(0, 99)]


### Арифметическое среднее

Разработайте my_map_reduce алгоритм, который находит арифметическое среднее.

$$\overline{X} = \frac{1}{n}\sum_{i=0}^{n} x_i$$


In [22]:
input = [3, 3, 3, 3, 8, 8, 8, 8]  # новые данные для примера


def record_reader():
    for idx, num in enumerate(input):
        yield str(idx), num


def my_map(num_id: str, num: int):
    """Аналогично предыдущему примеру: отправляем всё в один редьюсер."""
    yield 0, num


def my_reduce(dummy_key: str, nums: Iterator[int]):
    nums = list(nums)  # материализуем, чтобы посчитать len
    yield dummy_key, sum(nums) / len(nums)


print("AVERAGE:", list(my_map_reduce(record_reader, my_map, my_reduce)))  # [('0', 5.5)]

AVERAGE: [(0, 5.5)]


### GroupByKey на основе сортировки

Реализуйте groupByKey на основе сортировки, проверьте его работу на примерах

In [23]:
def groupbykey_sorted(pairs):
    """
    Принимает неупорядоченную последовательность кортежей (key, value),
    сортирует по key и вручную «собирает» списки значений для каждого key.
    Возвращает генератор (key, [value, value, …]).
    """
    pairs_sorted = sorted(pairs, key=lambda kv: kv[0])  # обязательная сортировка
    current_key = None  # ключ текущей «корзинки»
    bucket = []  # список значений, относящихся к current_key
    for key, val in pairs_sorted:
        if key != current_key and bucket:
            # Ключ сменился ⇒ старая корзинка закончена — отдаём наружу
            yield current_key, bucket
            bucket = []
        bucket.append(val)
        current_key = key

    # доотдаём «хвост», если он есть
    if bucket:
        yield current_key, bucket


# Несколько демонстрационных наборов
e1 = [('a', 1), ('b', 2), ('a', 3), ('b', 4), ('c', 5)]
e2 = [('x', 10), ('y', 20), ('x', 30), ('z', 40), ('y', 50)]
e3 = [('apple', 'red'), ('banana', 'yellow'),
      ('apple', 'green'), ('banana', 'green')]

print("GROUPBY #1:", list(groupbykey_sorted(e1)))
print("GROUPBY #2:", list(groupbykey_sorted(e2)))
print("GROUPBY #3:", list(groupbykey_sorted(e3)))

GROUPBY #1: [('a', [1, 3]), ('b', [2, 4]), ('c', [5])]
GROUPBY #2: [('x', [10, 30]), ('y', [20, 50]), ('z', [40])]
GROUPBY #3: [('apple', ['red', 'green']), ('banana', ['yellow', 'green'])]


### Drop duplicates (set construction, unique elements, distinct)

Реализуйте распределённую операцию исключения дубликатов

In [24]:
input_values = [2, 2, 2, 4, 5, 5, 6, 7, 8]  # исходные данные
maps, reducers = 3, 2  # параметризация кластера


def record_reader(split):
    """
    Читает «кусок» (split) входного массива и выдаёт (key, None),
    где key = само значение. None — потому что полезных данных,
    кроме ключа, нам не требуется.
    """
    for v in split:
        yield v, None


def input_format():
    """
    Делит input_values на `maps` примерно одинаковых частей
    и отдаёт итератор из record_reader-ов.
    """
    chunk = int(np.ceil(len(input_values) / maps))
    for pos in range(0, len(input_values), chunk):
        yield record_reader(input_values[pos:pos + chunk])


def my_map(v, _):
    yield v, None  # ключ = значение, полезная нагрузка не нужна


def partitioner(key, reducers=reducers):
    """
    Простейший хэш-разделитель: равномерно раскидываем ключи
    по `reducers` редьюсерам.
    """
    return hash(key) % reducers


def my_reduce(key, _):
    """
    До редьюсера доходит либо один экземпляр ключа,
    либо несколько (если дубликаты попали в тот же раздел) —
    в любом случае надо вывести ровно по одному key.
    """
    yield key, None


distinct_out = [(pid, list(data))
                for pid, data in map_reduce_distributed(input_format, my_map, my_reduce,
                                                        combiner=None,
                                                        partitioner=partitioner)]
print("DISTINCT partitions:", distinct_out)

9 key-value pairs were sent over a network.
DISTINCT partitions: [(0, [(2, None), (4, None), (6, None), (8, None)]), (1, [(5, None), (7, None)])]


# Операторы реляционной алгебры
### Selection (Выборка)

**The Map Function**: Для  каждого кортежа $t \in R$ вычисляется истинность предиката $C$. В случае истины создаётся пара ключ-значение $(t, t)$. В паре ключ и значение одинаковы, равны $t$.

**The Reduce Function:** Роль функции Reduce выполняет функция идентичности, которая возвращает то же значение, что получила на вход.



In [25]:
input = [13, 26, 37, 40, 42, 53, 60, 75, 88, 91]


def record_reader():
    for idx, num in enumerate(input):
        yield str(idx), num


def my_map(idx, num):
    """
    Если число чётное — пропускаем его дальше, иначе ничего не yield'им.
    """
    if num % 2 == 0:
        yield num, num


def my_reduce(num, duplicates):
    """
    Собираем дубликаты (они возможны, если вход содержал одно и то же число
    несколько раз) и оставляем список, чтобы увидеть «сколько раз встретилось».
    """
    yield num, list(duplicates)


print("EVEN filter:", list(my_map_reduce(record_reader, my_map, my_reduce)))

EVEN filter: [(26, [26]), (40, [40]), (42, [42]), (60, [60]), (88, [88])]


### Projection (Проекция)

Проекция на множество атрибутов $S$.

**The Map Function:** Для каждого кортежа $t \in R$ создайте кортеж $t′$, исключая  из $t$ те значения, атрибуты которых не принадлежат  $S$. Верните пару $(t′, t′)$.

**The Reduce Function:** Для каждого ключа $t′$, созданного любой Map задачей, вы получаете одну или несколько пар $(t′, t′)$. Reduce функция преобразует $(t′, [t′, t′, . . . , t′])$ в $(t′, t′)$, так, что для ключа $t′$ возвращается одна пара  $(t′, t′)$.

In [26]:
from typing import NamedTuple


class User(NamedTuple):
    id: int
    age: int
    social_contacts: int
    gender: str


users = [
    User(0, 44, 15, 'male'),
    User(1, 29, 300, 'female'),
    User(2, 31, 410, 'female'),
    User(3, 38, 910, 'female')
]
attributes = ['age', 'gender']  # поля, которые нужно оставить


def my_map(_, row: User):
    """
    Достаём нужные атрибуты (age, gender) и используем их и как ключ,
    и как значение (чтобы редьюсер увидел их «сырьём»).
    """
    proj = tuple(getattr(row, attr) for attr in attributes)
    yield proj, proj


def my_reduce(proj, _):
    """Никакой агрегации не нужно — просто повторяем ключ/значение."""
    yield proj, proj


def record_reader():
    return [(u.id, u) for u in users]


print("PROJECTION:", list(my_map_reduce(record_reader, my_map, my_reduce)))

PROJECTION: [((44, 'male'), (44, 'male')), ((29, 'female'), (29, 'female')), ((31, 'female'), (31, 'female')), ((38, 'female'), (38, 'female'))]


### Union (Объединение)

**The Map Function:** Превратите каждый входной кортеж $t$ в пару ключ-значение $(t, t)$.

**The Reduce Function:** С каждым ключом $t$ будет ассоциировано одно или два значений. В обоих случаях создайте $(t, t)$ в качестве выходного значения.

In [27]:
R = users  # «левая» коллекция
S = [
    User(0, 44, 1_500_000, 'male'),  # отличаются только social_contacts
    User(1, 29, 300, 'female'),
    User(2, 31, 410, 'female'),
    User(3, 38, 910, 'female')
]


def my_map(_, row):
    # ключом делаем сам объект User: одинаковые записи попадут в один редьюсер
    yield row, row


def my_reduce(row_key, _):
    """
    row_key — это уже уникальный объект User, дошедший до редьюсера.
    values нам не нужны: если таких объектов было несколько (дубликаты из R и S),
    my_map_reduce сгруппировал их по этому ключу.
    """
    yield row_key, None


def record_reader():
    return [(u.id, u) for u in R] + [(u.id, u) for u in S]


union_res = list(my_map_reduce(record_reader, my_map, my_reduce))
print("UNION size:", len(union_res))
print("UNION full:", union_res)

UNION size: 5
UNION full: [(User(id=0, age=44, social_contacts=15, gender='male'), None), (User(id=1, age=29, social_contacts=300, gender='female'), None), (User(id=2, age=31, social_contacts=410, gender='female'), None), (User(id=3, age=38, social_contacts=910, gender='female'), None), (User(id=0, age=44, social_contacts=1500000, gender='male'), None)]


### Intersection (Пересечение)

**The Map Function:** Превратите каждый кортеж $t$ в пары ключ-значение $(t, t)$.

**The Reduce Function:** Если для ключа $t$ есть список из двух элементов $[t, t]$ $-$ создайте пару $(t, t)$. Иначе, ничего не создавайте.

In [28]:
def my_map(_, row):
    yield row, row


def my_reduce(_, rows):
    rows = list(rows)
    if len(rows) == 2:  # встретился в обеих коллекциях
        yield rows[0], None


inter_res = list(my_map_reduce(record_reader, my_map, my_reduce))
print("INTERSECT:", inter_res)

INTERSECT: [(User(id=1, age=29, social_contacts=300, gender='female'), None), (User(id=2, age=31, social_contacts=410, gender='female'), None), (User(id=3, age=38, social_contacts=910, gender='female'), None)]


### Difference (Разница)

**The Map Function:** Для кортежа $t \in R$, создайте пару $(t, R)$, и для кортежа $t \in S$, создайте пару $(t, S)$. Задумка заключается в том, чтобы значение пары было именем отношения $R$ or $S$, которому принадлежит кортеж (а лучше, единичный бит, по которому можно два отношения различить $R$ or $S$), а не весь набор атрибутов отношения.

**The Reduce Function:** Для каждого ключа $t$, если соответствующее значение является списком $[R]$, создайте пару $(t, t)$. В иных случаях не предпринимайте действий.

In [29]:
def my_map(_, row, tag):
    """
    tag хранит маркер: 'R' если элемент пришёл из R, 'S' — из S.
    """
    yield row, tag


def my_reduce(row, tags):
    """
    Элемент остаётся, если присутствовал хотя бы один раз в R
    и ни разу в S.
    """
    tags = list(tags)
    if 'R' in tags and 'S' not in tags:
        yield row, None


def record_reader():
    return [(u.id, u, 'R') for u in R] + [(u.id, u, 'S') for u in S]


diff_res = list(my_map_reduce(record_reader, my_map, my_reduce))
print("R \\ S:", diff_res)

R \ S: [(User(id=0, age=44, social_contacts=15, gender='male'), None)]


### Natural Join

**The Map Function:** Для каждого кортежа $(a, b)$ отношения $R$, создайте пару $(b,(R, a))$. Для каждого кортежа $(b, c)$ отношения $S$, создайте пару $(b,(S, c))$.

**The Reduce Function:** Каждый ключ $b$ будет асоциирован со списком пар, которые принимают форму либо $(R, a)$, либо $(S, c)$. Создайте все пары, одни, состоящие из  первого компонента $R$, а другие, из первого компонента $S$, то есть $(R, a)$ и $(S, c)$. На выходе вы получаете последовательность пар ключ-значение из списков ключей и значений. Ключ не нужен. Каждое значение, это тройка $(a, b, c)$ такая, что $(R, a)$ и $(S, c)$ это принадлежат входному списку значений.

In [30]:
class Employee(NamedTuple):
    id: int
    salary: float


employees = [
    Employee(0, 350.0),
    Employee(1, 450.0),
    Employee(2, 2300.0),
    Employee(3, 7000.0)
]


def my_map(_, row, src):
    """
    src указывает источник ('U' — User, 'E' — Employee).
    В значение кладём кортеж (тип, полезные данные).
    """
    if src == 'U':
        yield row.id, ('U', row.gender)
    else:
        yield row.id, ('E', row.salary)


def my_reduce(id_, items):
    """
    Собираем всё под одинаковым id и формируем декартово произведение
    между User-полом и зарплатой Employee.
    """
    genders = [v for typ, v in items if typ == 'U']  # 'male' / 'female'
    salaries = [v for typ, v in items if typ == 'E']  # float
    for g in genders:
        for s in salaries:
            yield id_, g, s


def record_reader():
    return ([(u.id, u, 'U') for u in users] +
            [(e.id, e, 'E') for e in employees])


join_res = list(my_map_reduce(record_reader, my_map, my_reduce))
print("THETA JOIN:", join_res)

THETA JOIN: [(0, 'male', 350.0), (1, 'female', 450.0), (2, 'female', 2300.0), (3, 'female', 7000.0)]


### Grouping and Aggregation (Группировка и аггрегация)

**The Map Function:** Для каждого кортежа $(a, b, c$) создайте пару $(a, b)$.

**The Reduce Function:** Ключ представляет ту или иную группу. Примение аггрегирующую операцию $\theta$ к списку значений $[b1, b2, . . . , bn]$ ассоциированных с ключом $a$. Возвращайте в выходной поток $(a, x)$, где $x$ результат применения  $\theta$ к списку. Например, если $\theta$ это $SUM$, тогда $x = b1 + b2 + · · · + bn$, а если $\theta$ is $MAX$, тогда $x$ это максимальное из значений $b1, b2, . . . , bn$.

In [31]:
def my_map(_, row: User):
    yield row.gender, row.social_contacts


def my_reduce(gender, contacts, agg_op=sum):
    """
    contacts — итератор всех social_contacts для данного пола.
    Применяем выбранную агрегат-функцию
    """
    yield gender, agg_op(contacts)


def record_reader():
    return [(u.id, u) for u in users]


agg_res = list(my_map_reduce(record_reader, my_map, my_reduce))
print("AGGREGATE:", agg_res)

AGGREGATE: [('male', 15), ('female', 1620)]


#

### Matrix-Vector multiplication

Случай, когда вектор не помещается в памяти Map задачи


In [32]:
import numpy as np

A = np.random.randint(1, 5, size=(5, 4))  # случайная матрица 5×4
v = np.random.rand(4)  # длина 4


def my_map(coords, val):
    """
    coords = (i, j)
    Для каждой позиции (i,j) создаём пару (i, (j, val)),
    где val -- либо A[i, j], либо v[j] (подмешивается в record_reader).
    """
    i, j = coords
    yield i, (j, val)


def my_reduce(i, pairs):
    """
    pairs содержит и элементы строки A[i,*], и все v[*].
    Для корректности считаем произведение только там,
    где «под нужным j» встретилось ровно два значения — A_ij и v_j.
    """
    cache = {}
    for j, x in pairs:
        cache.setdefault(j, []).append(x)
    dot = 0.0
    for j, two_vals in cache.items():
        if len(two_vals) == 2:
            dot += two_vals[0] * two_vals[1]
    yield i, float(dot)


def record_reader():
    """
    Пробегаем все координаты матрицы:
      - публикуем A[i,j]
      - сразу же публикуем v[j] с тем же ключом (i,j)
    В итоге и A, и v попадают в my_map с одинаковыми coords.
    """
    for i in range(A.shape[0]):
        for j in range(A.shape[1]):
            yield (i, j), A[i, j]
            yield (i, j), v[j]


mv_res = list(my_map_reduce(record_reader, my_map, my_reduce))
print("MAT×VEC:", mv_res)

MAT×VEC: [(0, 3.0142366538715084), (1, 7.645696316137098), (2, 5.932406207646883), (3, 5.626846825981297), (4, 4.492509837857999)]


## Matrix multiplication (Перемножение матриц)

Если у нас есть матрица $M$ с элементами $m_{ij}$ в строке $i$ и столбце $j$, и матрица $N$ с элементами $n_{jk}$ в строке $j$ и столбце $k$, тогда их произведение $P = MN$ есть матрица $P$ с элементами $p_{ik}$ в строке $i$ и столбце $k$, где

$$p_{ik} =\sum_{j} m_{ij}n_{jk}$$

Необходимым требованием является одинаковое количество столбцов в $M$ и строк в $N$, чтобы операция суммирования по  $j$ была осмысленной. Мы можем размышлять о матрице, как об отношении с тремя атрибутами: номер строки, номер столбца, само значение. Таким образом матрица $M$ предстваляется как отношение $ M(I, J, V )$, с кортежами $(i, j, m_{ij})$, и, аналогично, матрица $N$ представляется как отношение $N(J, K, W)$, с кортежами $(j, k, n_{jk})$. Так как большие матрицы как правило разреженные (большинство значений равно 0), и так как мы можем нулевыми значениями пренебречь (не хранить), такое реляционное представление достаточно эффективно для больших матриц. Однако, возможно, что координаты $i$, $j$, и $k$ неявно закодированы в смещение позиции элемента относительно начала файла, вместо явного хранения. Тогда, функция Map (или Reader) должна быть разработана таким образом, чтобы реконструировать компоненты $I$, $J$, и $K$ кортежей из смещения.

Произведение $MN$ это фактически join, за которым следуют группировка по ключу и аггрегация. Таким образом join отношений $M(I, J, V )$ и $N(J, K, W)$, имеющих общим только атрибут $J$, создаст кортежи $(i, j, k, v, w)$ из каждого кортежа $(i, j, v) \in M$ и кортежа $(j, k, w) \in N$. Такой 5 компонентный кортеж представляет пару элементов матрицы $(m_{ij} , n_{jk})$. Что нам хотелось бы получить на самом деле, это произведение этих элементов, то есть, 4 компонентный кортеж$(i, j, k, v \times w)$, так как он представляет произведение $m_{ij}n_{jk}$. Мы представляем отношение как результат одной my_map_reduce операции, в которой мы можем произвести группировку и аггрегацию, с $I$ и $K$  атрибутами, по которым идёт группировка, и суммой  $V \times W$.





In [33]:
# my_map_reduce model
def flatten(nested_iterable):
    for iterable in nested_iterable:
        for element in iterable:
            yield element


def group_by_key(iterable):
    t = {}
    for (k2, v2) in iterable:
        t[k2] = t.get(k2, []) + [v2]
    return t.items()


def my_map_reduce(record_reader, my_map, my_reduce):
    return flatten(map(lambda x: my_reduce(*x), group_by_key(flatten(map(lambda x: my_map(*x), record_reader())))))

Реализуйте перемножение матриц с использованием модельного кода my_map_reduce для одной машины в случае, когда одна матрица хранится в памяти, а другая генерируется record_reader-ом.

In [34]:
import numpy as np

I = 2
J = 3
K = 4 * 10
small_mat = np.random.rand(I, J)  # it is legal to access this from record_reader, my_map, my_reduce
big_mat = np.random.rand(J, K)


def record_reader():
    for j in range(big_mat.shape[0]):
        for k in range(big_mat.shape[1]):
            yield (j, k), big_mat[j, k]


def my_map(k1, v1):
    (j, k) = k1
    w = v1
    for i in range(I):
        yield (i, k), (w * small_mat[i, j])


def my_reduce(key, values):
    yield key, sum(values)

Проверьте своё решение

In [35]:
reference_solution = np.matmul(small_mat, big_mat)
solution = my_map_reduce(record_reader, my_map, my_reduce)


def asmatrix(reduce_output):
    reduce_output = list(reduce_output)
    a = max(i for ((i, k), vw) in reduce_output) + 1
    b = max(k for ((i, k), vw) in reduce_output) + 1
    mat = np.empty(shape=(a, b))
    for ((i, k), vw) in reduce_output:
        mat[i, k] = vw
    return mat


np.allclose(reference_solution, asmatrix(solution))  # should return true

True

In [36]:
reduce_output = list(my_map_reduce(record_reader, my_map, my_reduce))
max(i for ((i, k), vw) in reduce_output)

1

Реализуйте перемножение матриц  с использованием модельного кода my_map_reduce для одной машины в случае, когда обе матрицы генерируются в record_reader. Например, сначала одна, а потом другая.

In [37]:
import numpy as np


# -----------------------------------------------------------
# вспомогательные утилиты форматирования
# -----------------------------------------------------------
def tolist2d(reduce_out):
    """[( (i,k), value ), … ]  →  2-D list  [[...], [...], ...]"""
    if isinstance(reduce_out[0][0], tuple):  # обычный вариант
        a = max(i for (i, _), _v in reduce_out) + 1
        b = max(k for (_, k), _v in reduce_out) + 1
        mat = [[0.0] * b for _ in range(a)]
        for (i, k), v in reduce_out:
            mat[i][k] = v
        return mat
    else:  # distributed (partition, list)
        flat = []
        for _pid, data in reduce_out:
            flat.extend(data)
        return tolist2d(flat)


def print_matrix(mat, name):
    print(f"{name} ({len(mat)}×{len(mat[0])}):")
    for row in mat:
        print("  ", " ".join(f"{x:8.4f}" for x in row))
    print()


# ============================================================
# Полное умножение M × N (одна машина)
# ============================================================
I, J, K = 2, 3, 12
M = np.random.rand(I, J)
N = np.random.rand(J, K)


def record_reader():
    for j in range(J):
        for k in range(K):
            yield (j, k), N[j, k], 'N'

        for i in range(I):
            yield (i, j), M[i, j], 'M'


def my_map(coord, val, tag):
    if tag == 'N':
        j, k = coord
        for i in range(I):
            yield (i, k), (j, val)

    else:  # tag == 'M'
        i, j = coord
        for k in range(K):
            yield (i, k), (j, val)


def my_reduce(idx, pairs):
    bucket = {}

    for j, x in pairs:
        bucket.setdefault(j, []).append(x)
    s = 0.0

    for j, lst in bucket.items():
        if len(lst) == 2:
            s += lst[0] * lst[1]
    yield idx, float(s)


matmul_pairs = list(my_map_reduce(record_reader, my_map, my_reduce))
matmul_matrix = np.array(tolist2d(matmul_pairs))
print_matrix(matmul_matrix, "MAT×MAT result")

# быстрая проверка
print("check :", np.allclose(matmul_matrix, M @ N), "\n")

MAT×MAT result (2×12):
     0.5919   1.0228   0.9136   0.7980   0.5115   1.0181   0.5508   0.8271   0.7304   0.6284   0.4280   0.6614
     0.5685   0.9828   0.8950   0.8435   0.4969   1.0520   0.5382   0.8056   0.6416   0.6949   0.3754   0.6281

check : True 



Реализуйте перемножение матриц с использованием модельного кода my_map_reduce Distributed, когда каждая матрица генерируется в своём record_reader.

In [38]:
maps, reducers = 2, 2


def record_reader(i_max, j_max, tag):
    if tag == 'M':
        for i in range(I):
            for j in range(J):
                yield (i, j), M[i, j], 'M'
    else:
        for j in range(J):
            for k in range(K):
                yield (j, k), N[j, k], 'N'


def input_format():
    yield record_reader(I, J, 'M')
    yield record_reader(J, K, 'N')


dist1 = [(pid, list(part))  # материализуем генератор
         for pid, part in map_reduce_distributed(
        input_format, my_map, my_reduce,
        combiner=None, partitioner=partitioner)]

matmul_matrix_d1 = np.array(tolist2d(dist1))
print_matrix(matmul_matrix_d1, "distributed #1")

print("check :", np.allclose(matmul_matrix_d1, M @ N), "\n")

144 key-value pairs were sent over a network.
distributed #1 (2×12):
     0.5919   1.0228   0.9136   0.7980   0.5115   1.0181   0.5508   0.8271   0.7304   0.6284   0.4280   0.6614
     0.5685   0.9828   0.8950   0.8435   0.4969   1.0520   0.5382   0.8056   0.6416   0.6949   0.3754   0.6281

check : True 



Обобщите предыдущее решение на случай, когда каждая матрица генерируется несколькими record_reader-ами, и проверьте его работоспособность. Будет ли работать решение, если record_reader-ы будут генерировать случайное подмножество элементов матрицы?

In [39]:
maps, reducers = 5, 3
I, J, K = 2, 3, 12
M = np.random.rand(I, J)
N = np.random.rand(J, K)


def record_reader(idx_subset, tag):
    for (i, j) in idx_subset:
        if tag == 'M':
            yield (i, j), M[i, j], 'M'
        else:
            yield (i, j), N[i, j], 'N'


def input_format():
    M_idx = [(i, j) for i in range(I) for j in range(J)]
    N_idx = [(j, k) for j in range(J) for k in range(K)]
    np.random.shuffle(M_idx)
    np.random.shuffle(N_idx)

    M_chunk = len(M_idx) // (maps // 2)
    N_chunk = len(N_idx) // (maps - maps // 2)

    for pos in range(0, len(M_idx), M_chunk):
        yield record_reader(M_idx[pos:pos + M_chunk], 'M')
    for pos in range(0, len(N_idx), N_chunk):
        yield record_reader(N_idx[pos:pos + N_chunk], 'N')


dist2 = [(pid, list(part))
         for pid, part in map_reduce_distributed(
        input_format, my_map, my_reduce,
        combiner=None, partitioner=partitioner)]

matmul_matrix_d2 = np.array(tolist2d(dist2))
print_matrix(matmul_matrix_d2, "distributed #2")

print("check :", np.allclose(matmul_matrix_d2, M @ N))

144 key-value pairs were sent over a network.
distributed #2 (2×12):
     0.6915   0.5360   1.5806   0.9690   0.8718   0.8431   1.4671   1.7091   0.7124   0.7674   0.8498   0.6003
     0.5445   0.5935   1.1766   0.9324   0.8159   0.8013   1.1598   1.3969   0.9126   0.4123   0.5338   0.5215

check : True
